In [9]:
"""
The Beer Distribution Problem for the PuLP Modeller

Authors: Antony Phillips, Dr Stuart Mitchell    2007
"""

# Import PuLP modeller functions
from pulp import *

In [16]:
# Creates a list of all the supply nodes
Warehouses = ["A","B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000,
        "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1": 500,
        "2": 900,
        "3": 1800,
        "4": 200,
        "5": 700}

costs = {"A":[0,2,4,5,2,1],#A  Warehouses
        "B":[0,3,1,3,2,3] #B
}


In [17]:
costs

{'A': [0, 2, 4, 5, 2, 1], 'B': [0, 3, 1, 3, 2, 3]}

In [18]:
# Creates the prob variable to contain the problem data
prob = LpProblem("Beer Distribution Problem",LpMinimize)
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Warehouses for b in Bars]
print(Routes)
# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(Warehouses,Bars),0,None,LpInteger)
print(route_vars)

[('A', '1'), ('A', '2'), ('A', '3'), ('A', '4'), ('A', '5'), ('B', '1'), ('B', '2'), ('B', '3'), ('B', '4'), ('B', '5')]
{'A': {'1': Route_A_1, '2': Route_A_2, '3': Route_A_3, '4': Route_A_4, '5': Route_A_5}, 'B': {'1': Route_B_1, '2': Route_B_2, '3': Route_B_3, '4': Route_B_4, '5': Route_B_5}}


In [22]:
# The objective function is added to prob first
prob += lpSum([route_vars[w][b]*costs[w][int(b)] for (w,b) in Routes]), "Sum of Transporting Costs"

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += lpSum([route_vars[w][b] for b in Bars]) <= supply[w], "Sum of Products out of Warehouse %s"%w

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += lpSum([route_vars[w][b] for w in Warehouses]) >= demand[b], "Sum of Products into Bars %s"%b

In [26]:
print(prob)

Beer Distribution Problem:
MINIMIZE
2*Route_A_1 + 4*Route_A_2 + 5*Route_A_3 + 2*Route_A_4 + 1*Route_A_5 + 3*Route_B_1 + 1*Route_B_2 + 3*Route_B_3 + 2*Route_B_4 + 3*Route_B_5 + 0
SUBJECT TO
Sum_of_Products_out_of_Warehouse_A: Route_A_1 + Route_A_2 + Route_A_3
 + Route_A_4 + Route_A_5 <= 1000

Sum_of_Products_out_of_Warehouse_B: Route_B_1 + Route_B_2 + Route_B_3
 + Route_B_4 + Route_B_5 <= 4000

Sum_of_Products_into_Bars_1: Route_A_1 + Route_B_1 >= 500

Sum_of_Products_into_Bars_2: Route_A_2 + Route_B_2 >= 900

Sum_of_Products_into_Bars_3: Route_A_3 + Route_B_3 >= 1800

Sum_of_Products_into_Bars_4: Route_A_4 + Route_B_4 >= 200

Sum_of_Products_into_Bars_5: Route_A_5 + Route_B_5 >= 700

VARIABLES
0 <= Route_A_1 Integer
0 <= Route_A_2 Integer
0 <= Route_A_3 Integer
0 <= Route_A_4 Integer
0 <= Route_A_5 Integer
0 <= Route_B_1 Integer
0 <= Route_B_2 Integer
0 <= Route_B_3 Integer
0 <= Route_B_4 Integer
0 <= Route_B_5 Integer



In [27]:
def print_rozwiazanie(p):
    p.solve()
    print("Status:", LpStatus[p.status])
    for v in p.variables():
        print (v.name, "=", v.varValue)
    print("Cel = ", value(p.objective))


In [28]:
print_rozwiazanie(prob)

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Cel =  8600.0
